In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [2]:
game = pd.read_csv('../data/clean_game_data.csv')
game
#DATAFRAME CLEANSING (STANDARDIZED NAMES) NEEDED

,Unnamed: 0,season,week,home_team,away_team,home.totalYards,away.totalYards,Home_possession_time,Away_possession_time,Home_Offensive_TDs,Away_Offensive_TDs,home.turnovers,away.turnovers,home.thirdDownEff,home.fourthDownEff,away.thirdDownEff,away.fourthDownEff,away_points,home_points
0,0,2013,1,Minnesota,UNLV,320,419,1286,2314,4,3.0,1,2,0.571429,1.00,0.333333,0.333333,23.0,51.0
1,1,2013,1,Indiana,Indiana State,632,306,1286,2314,9,3.0,3,2,0.500000,0.50,0.187500,0.187500,35.0,73.0
2,2,2013,1,Michigan State,Western Michigan,297,204,1286,2314,1,2.0,1,4,0.263158,0.00,0.200000,0.200000,13.0,26.0
3,3,2013,1,Cincinnati,Purdue,425,226,1286,2314,5,1.0,3,4,0.600000,0.75,0.333333,0.333333,7.0,42.0
4,4,2013,1,Wisconsin,UMass,598,212,1286,2314,6,0.0,1,2,0.727273,0.00,0.428571,0.428571,0.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,840,2021,4,Maryland,Kent State,528,458,1286,2314,5,1.0,2,1,0.363636,0.50,0.250000,0.250000,16.0,37.0
839,841,2021,4,Michigan,Rutgers,275,352,1286,2314,2,1.0,0,1,0.363636,0.00,0.437500,0.437500,13.0,20.0
840,842,2021,4,Michigan State,Nebraska,254,444,1286,2314,1,2.0,1,2,0.100000,0.00,0.368421,0.368421,20.0,23.0
841,843,2021,4,Ohio State,Akron,622,229,1286,2314,7,1.0,1,2,0.727273,0.00,0.277778,0.277778,7.0,59.0


In [3]:
game["point_diff"] = game["home_points"] - game["away_points"]
game["home_win"] = game["point_diff"] > 0

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
X = game[["season","home.totalYards","away.totalYards","Home_possession_time","Away_possession_time","Home_Offensive_TDs","Away_Offensive_TDs","home.turnovers","away.turnovers","home.thirdDownEff","home.fourthDownEff","away.thirdDownEff","away.fourthDownEff"]].dropna()
#X = game[["season","home.totalYards","Home_possession_time","Home_Offensive_TDs","home.turnovers"]].dropna()

In [5]:
X_train = X[X["season"] < 2021].drop(columns = ["season"])
X_test = X[X["season"] >= 2021].drop(columns = ["season"])
y_train = game.loc[X_train.index]["point_diff"].dropna()
y_test = game.loc[X_test.index]["point_diff"].dropna()

In [6]:
model = linear_model.Lasso(alpha=0.1)
model.fit(X_train, y_train)

Lasso(alpha=0.1)

In [7]:
y_pred = model.predict(X_test)
print(model.intercept_)
print(model.coef_)
print(model.score(X_test, y_test))
print(np.mean(model.predict(X_test)-y_test))
print(np.mean((model.predict(X_test)-y_test)**2)**0.5)

1.1708700199813622
[ 0.02338547 -0.02634921  0.          0.          5.62627715 -5.56403556
 -1.65020134  1.82138652  0.          0.46548929 -0.         -0.        ]
0.9260686581063149
-0.6817599081599849
6.193401123110496


In [8]:
penn_2021 = game[game["season"] == 2021]
penn_2021 = penn_2021[penn_2021["home_team"] == "Penn State"]
michigan_2021 = game[game["season"] == 2021]
michigan_2021 = michigan_2021[michigan_2021["home_team"] == "Michigan"]
penn_2021

,Unnamed: 0,season,week,home_team,away_team,home.totalYards,away.totalYards,Home_possession_time,Away_possession_time,Home_Offensive_TDs,...,home.turnovers,away.turnovers,home.thirdDownEff,home.fourthDownEff,away.thirdDownEff,away.fourthDownEff,away_points,home_points,point_diff,home_win
813,815,2021,2,Penn State,Ball State,493,295,1286,2314,4,...,0,2,0.461538,0.0,0.250000,0.250000,13.0,44.0,31.0,True
831,833,2021,3,Penn State,Auburn,386,367,1286,2314,4,...,1,1,0.500000,0.0,0.588235,0.588235,20.0,28.0,8.0,True
834,836,2021,4,Penn State,Villanova,509,280,1286,2314,5,...,1,1,0.250000,1.0,0.285714,0.285714,17.0,38.0,21.0,True


In [9]:
michigan_2021 = michigan_2021[["home.totalYards","Home_possession_time","Home_Offensive_TDs","home.turnovers","home.thirdDownEff","home.fourthDownEff"]].mean()
penn_2021 = penn_2021[["home.totalYards","away.totalYards","Home_possession_time","Away_possession_time","Home_Offensive_TDs","Away_Offensive_TDs","home.turnovers","away.turnovers","home.thirdDownEff","home.fourthDownEff","away.thirdDownEff","away.fourthDownEff"]].mean()

In [10]:
penn_2021["away.totalYards"] = michigan_2021["home.totalYards"]
penn_2021["Away_possession_time"] = michigan_2021["Home_possession_time"]
penn_2021["Away_Offensive_TDs"] = michigan_2021["Home_Offensive_TDs"]
penn_2021["away.turnovers"] = michigan_2021["home.turnovers"]
penn_2021["away.thirdDownEff"] = michigan_2021["home.thirdDownEff"]
penn_2021["away.fourthDownEff"] = michigan_2021["home.fourthDownEff"]

X_test.iloc[0]

home.totalYards          326.000000
away.totalYards          392.000000
Home_possession_time    1286.000000
Away_possession_time    2314.000000
Home_Offensive_TDs         3.000000
Away_Offensive_TDs         3.000000
home.turnovers             1.000000
away.turnovers             1.000000
home.thirdDownEff          0.461538
home.fourthDownEff         0.000000
away.thirdDownEff          0.357143
away.fourthDownEff         0.357143
Name: 796, dtype: float64

In [11]:

penn_2021

home.totalYards          462.666667
away.totalYards          454.500000
Home_possession_time    1286.000000
Away_possession_time    1286.000000
Home_Offensive_TDs         4.333333
Away_Offensive_TDs         5.250000
home.turnovers             0.666667
away.turnovers             0.000000
home.thirdDownEff          0.403846
home.fourthDownEff         0.333333
away.thirdDownEff          0.491951
away.fourthDownEff         0.500000
dtype: float64

In [12]:
pennvmich = pd.DataFrame(penn_2021)

In [13]:
model.predict(pennvmich.transpose())

array([-5.7607916])